# Proposition 8: Simulations with Original Program (Summary)

### Unified Model, Multiple Weighted Strategic Agents, Symmetric Network

James Yu, 22 July 2023

In [1]:
from collections import defaultdict
import matplotlib.pyplot as plt
import numpy as np
np.set_printoptions(suppress=True)

## Unprojected Solution Code

In [2]:
def M(K, B, R, L, delta):
    """Computes M_{t-1} given B_l \forall l, K_t^l \forall l, 
        R_l \forall l, number of strategic agents L, and delta."""
    # handle the generic structure first, with the correct pairings:
    base = [[B[l_prime].T @ K[l_prime] @ B[l] for l in range(L)] for l_prime in range(L)]
    # then change the diagonals to construct M_{t-1}:
    for l in range(L): base[l][l] = B[l].T @ K[l] @ B[l] + R[l]/delta
    return np.block(base)

def H(B, K, A, L):
    """Computes H_{t-1} given B_l \forall l, K_t^l \forall l, 
        A, and number of strategic agents L."""
    return np.concatenate(tuple(B[l].T @ K[l] @ A for l in range(L)), axis = 0)

def C_l(A, B, K, k, h, L, c, x, n):
    """Computes C_{t-1}^h (displayed as C_{t-1}^l) given A, B_l \forall l, K_t^l \forall l, 
        k_t^l \forall l, a specific naive agent h, number of strategic agents L, 
        c_l \forall l, x_l \forall l, and number of naive agents n"""
    return np.concatenate(tuple(B[l].T @ K[l] @ A @ ((x[h] - x[l]) * np.ones((n, 1))) 
                           + B[l].T @ K[l] @ c[l] 
                           + 0.5 * B[l].T @ k[l].T for l in range(L)), axis = 0)

def E(M_, H_):
    """Computes the generic E_{t-1} given M_{t-1} and H_{t-1}."""
    return np.linalg.inv(M_) @ H_

def F(M_, C_l_, l, n):
    """Computes F_{t-1}^l given M_{t-1}, C_{t-1}^l, 
       specific naive agent l and number of naive agents n."""
    return (np.linalg.inv(M_) @ C_l_)[l*n:(l+1)*n, :] # e.g. l = 0 gives ln = 0, l = 1 gives ln = n, etc

def G(A, B, E_, L, n):
    """Computes the generic G_{t-1} given A, B_l \forall l, 
        E_{t-1}, number of strategic agents L, and number of naive agents n."""
    return A - sum([B[l] @ E_[l*n:(l+1)*n, :] for l in range(L)])
    
def g_l(B, E_, h, x, F_, L, n, c):
    """Computes g_{t-1}^l given B_l \forall l, E_{t-1}^l, 
        a particular naive agent h, x_l \forall l, F_{t-1}^l \forall l, 
        number of strategic agents L, number of naive agents n, and c_h."""
    return - sum([B[l] @ (E_[l*n:(l+1)*n, :] @ ((x[h] - x[l]) * np.ones((n, 1))) + F_[l]) for l in range(L)]) + c[h]

In [3]:
def K_t_minus_1(Q, K, E_, R, G_, L, delta, n):
    return [Q[l] + E_[l*n:(l+1)*n, :].T @ R[l] @ E_[l*n:(l+1)*n, :] 
            + delta * G_.T @ K[l] @ G_ for l in range(L)]

def k_t_minus_1(K, k, G_, g, E_, F_, R, L, delta, n):
    return [2*delta* g[l].T @ K[l] @ G_ + delta * k[l] @ G_ 
            + 2 * F_[l].T @ R[l] @ E_[l*n:(l+1)*n, :] for l in range(L)]

def kappa_t_minus_1(K, k, kappa, g_, F_, R, L, delta):            
    return [-delta * (g_[l].T @ K[l] @ g_[l] + k[l] @ g_[l] - kappa[l]) 
            - (F_[l].T @ R[l] @ F_[l]) for l in range(L)]

In [4]:
def should_terminate(bundles, eps):
    return all([np.allclose(b[0], b[1], rtol = eps, atol = eps) for b in bundles])

def solve(K_t, k_t, kappa_t, A, B, delta, n, L, Q, R, x, c, tol):
    historical_K = [K_t]
    historical_k = [k_t]
    historical_kappa = [kappa_t]
    eps = np.sqrt(10**-11)
    while True:
        M_ = M(K_t, B, R, L, delta)
        H_ = H(B, K_t, A, L)
        E_ = E(M_, H_)
        G_ = G(A, B, E_, L, n)
        K_new = K_t_minus_1(Q, K_t, E_, R, G_, L, delta, n)
        F_ = [F(M_, C_l(A, B, K_t, k_t, l, L, c, x, n), l, n) for l in range(L)]
        g = [g_l(B, E_, h, x, F_, L, n, c) for h in range(L)]
        k_new = k_t_minus_1(K_t, k_t, G_, g, E_, F_, R, L, delta, n)
        kappa_new = kappa_t_minus_1(K_t, k_t, kappa_t, g, F_, R, L, delta)
        historical_K.insert(0, K_new)
        historical_k.insert(0, k_new)
        historical_kappa.insert(0, kappa_new)
        if should_terminate([(K_t, K_new), (k_t, k_new), (kappa_t, kappa_new)], eps):
            return historical_K[0], historical_k[0], historical_kappa[0]
        K_t = K_new
        k_t = k_new
        kappa_t = kappa_new

In [5]:
def optimal(X_init, K_ss, k_ss, A, B, delta, n, L, Q, R, x, c, eps):
    X_t = [a.copy() for a in X_init]
    xs = defaultdict(list)
    for l in range(L):
        xs[l].append(X_t[l])
        
    rs = defaultdict(list)
    payoffs = defaultdict(list)
    payoff = defaultdict(int)
    
    M_ = M(K_ss, B, R, L, delta)
    H_ = H(B, K_ss, A, L)
    E_ = E(M_, H_)
    G_ = G(A, B, E_, L, n)
    F_ = [F(M_, C_l(A, B, K_ss, k_ss, l, L, c, x, n), l, n) for l in range(L)]
    g = [g_l(B, E_, h, x, F_, L, n, c) for h in range(L)]
    
    i = 0
    while True:
        for l in range(L):
            # TODO: special code for finite horizon needs a terminal time T term
            Y_new = -1 * E_[l*n:(l+1)*n, :] @ X_t[l] - F_[l]
            rs[l].append(Y_new)
            payoff[l] += (-1 * delta**i * (X_t[l].T @ Q[l] @ X_t[l])).item() + (-1 * delta**i * (Y_new.T @ R[l] @ Y_new)).item()
            payoffs[l].append(payoff[l])
            X_new = G_ @ X_t[l] + g[l]
            xs[l].append(X_new)
            if l == L - 1 and np.allclose(X_t[l], X_new, rtol = eps, atol = eps):
                return xs, rs, payoffs
            X_t[l] = X_new 
        i += 1
        
    return xs, rs, payoffs

In [6]:
def run_simulation(A, b1, b2, alpha1, alpha2, delta, c, X_0_1, tol = 10**-11):
    X_0 = [X_0_1 - b1, X_0_1 - b2]
    n = A.shape[0] # number of naive agents
    L = 2 # number of strategic agents
    Q = [np.identity(n), np.identity(n)] # objective function for messages is just X'IX = X'X
    R = [c * np.identity(n), c * np.identity(n)] # message cost R = cI_n for some c under the new notation
    B = [alpha1 * np.identity(n), alpha2 * np.identity(n)] # B^l = a_l I_n
    x = [b1, b2] # agendas
    r = [0, 0] # message cost minimality is centered around zero
    c_base = sum([B[l] @ (r[l] * np.ones((n, 1))) for l in range(L)])
    c = [c_base + (A - np.identity(n)) @ (x[l] * np.ones((n, 1))) for l in range(L)] # normalization vector
    
    K_ss, k_ss, kappa_ss = solve(Q, [np.zeros((1, n)), np.zeros((1, n))], [0, 0], A, B, delta, n, L, Q, R, x, c, tol)
    xs, rs, payoffs = optimal(X_0, K_ss, k_ss, A, B, delta, n, L, Q, R, x, c, tol)
    
    result = {
        "K": K_ss,
        "k": k_ss,
        "kappa": kappa_ss,
        "x": xs,
        "r": rs,
        "payoffs": payoffs
    }
    return result

# Common parameters for the examples:

In [7]:
delta = 0.9
c = 2.0
x0 = np.array([[10.0, -5.0]], ndmin = 2).T
b1 = 10
b2 = -5
alpha1 = 6
alpha2 = 5

# Example 1: $a_1 = 0.6, a_2 = 0.4$

In [8]:
A1 = np.array([
    [0.6, 0.4],
    [0.6, 0.4]
])
V1 = (1/np.sqrt(0.6**2 + 0.4**2)) * np.array([
    [0.6, -0.4],
    [0.4, 0.6]
])

In [9]:
res1 = run_simulation(A1, b1, b2, alpha1, alpha2, delta, c, x0) # compute the solution

In [10]:
tilde_K1_example1 = (V1.T @ res1["K"][0] @ V1) # tilde K1
tilde_k1_example1 = np.linalg.inv(2 * V1 @ tilde_K1_example1) @ (2 * (b1 * np.ones((2, 1))).T @ res1["K"][0] - res1["k"][0]).T # tilde_k1
tilde_K2_example1 = (V1.T @ res1["K"][1] @ V1) # tilde K2
tilde_k2_example1 = np.linalg.inv(2 * V1 @ tilde_K2_example1) @ (2 * (b2 * np.ones((2, 1))).T @ res1["K"][1] - res1["k"][1]).T # tilde_k2
tilde_x_infty_example1 = V1.T @ (res1["x"][0][-1] + 10) # projected limit opinions
tilde_r_1_example1 = [V1.T @ r for r in res1["r"][0]] # projected messages, influencer 1
tilde_r_2_example1 = [V1.T @ r for r in res1["r"][1]] # projected messages, influencer 2

In [11]:
tilde_K1_example1 # tilde_K1

array([[ 1.01998279, -0.        ],
       [ 0.        ,  1.        ]])

In [12]:
tilde_K2_example1 # tilde_K2

array([[ 1.01407389, -0.        ],
       [-0.        ,  1.        ]])

In [13]:
tilde_k1_example1 # tilde_k1

array([[22.36291019],
       [ 2.77350098]])

In [14]:
tilde_k2_example1 # tilde_k2

array([[-15.59902881],
       [ -1.38675049]])

In [15]:
tilde_x_infty_example1 # projected limit opinions

array([[6.85805691],
       [1.07913477]])

In [16]:
tilde_r_1_example1 # projected messages, influencer 1

[array([[42.82444069],
        [ 4.59967346]]),
 array([[42.70395877],
        [ 4.57564887]]),
 array([[42.69979447],
        [ 4.57481849]]),
 array([[42.69965053],
        [ 4.57478979]]),
 array([[42.69964556],
        [ 4.5747888 ]]),
 array([[42.69964539],
        [ 4.57478877]]),
 array([[42.69964538],
        [ 4.57478877]]),
 array([[42.69964538],
        [ 4.57478877]])]

In [17]:
tilde_r_2_example1 # projected messages, influencer 2

[array([[-51.13618083],
        [ -5.52750459]]),
 array([[-51.23600079],
        [ -5.54752508]]),
 array([[-51.23945094],
        [ -5.54821707]]),
 array([[-51.23957019],
        [ -5.54824098]]),
 array([[-51.23957431],
        [ -5.54824181]]),
 array([[-51.23957445],
        [ -5.54824184]]),
 array([[-51.23957446],
        [ -5.54824184]]),
 array([[-51.23957446],
        [ -5.54824184]])]

In [18]:
res1["payoffs"][0] # cumulative payoffs at each stage, influencer 1

[-3935.1794328124647,
 -7302.800503748285,
 -10332.548075650906,
 -13059.286370781792,
 -15513.349762643142,
 -17722.00678191668,
 -19709.798098223826,
 -21498.810282867937]

In [19]:
res1["payoffs"][1] # cumulative payoffs at each stage, influencer 2

[-5515.924594077803,
 -10472.05085078967,
 -14934.160961482312,
 -18950.109775987745,
 -22564.46525559832,
 -25817.385235357175,
 -28745.013218636697,
 -31379.878403634822]

# Example 2: $a_1 = 0.7, a_2 = 0.3$

In [20]:
A2 = np.array([
    [0.7, 0.3],
    [0.7, 0.3]
])
V2 = (1/np.sqrt(0.7**2 + 0.3**2)) * np.array([
    [0.7, -0.3],
    [0.3, 0.7]
])

In [21]:
res2 = run_simulation(A2, b1, b2, alpha1, alpha2, delta, c, x0) # compute the solution

In [22]:
tilde_K1_example2 = (V2.T @ res2["K"][0] @ V2) # tilde K1
tilde_k1_example2 = np.linalg.inv(2 * V2 @ tilde_K1_example2) @ (2 * (b1 * np.ones((2, 1))).T @ res2["K"][0] - res2["k"][0]).T # tilde_k1
tilde_K2_example2 = (V2.T @ res2["K"][1] @ V2) # tilde K2
tilde_k2_example2 = np.linalg.inv(2 * V2 @ tilde_K2_example2) @ (2 * (b2 * np.ones((2, 1))).T @ res2["K"][1] - res2["k"][1]).T # tilde_k2
tilde_x_infty_example2 = V2.T @ (res2["x"][0][-1] + 10) # projected limit opinions
tilde_r_1_example2 = [V2.T @ r for r in res2["r"][0]] # projected messages, influencer 1
tilde_r_2_example2 = [V2.T @ r for r in res2["r"][1]] # projected messages, influencer 2

In [23]:
tilde_K1_example2 # tilde_K1

array([[ 1.0222883, -0.       ],
       [-0.       ,  1.       ]])

In [24]:
tilde_K2_example2 # tilde_K2

array([[1.01569802, 0.        ],
       [0.        , 1.        ]])

In [25]:
tilde_k1_example2 # tilde_k1

array([[22.08254466],
       [ 5.25225731]])

In [26]:
tilde_k2_example2 # tilde_k2

array([[-15.69957821],
       [ -2.62612866]])

In [27]:
tilde_x_infty_example2 # projected limit opinions

array([[6.65713561],
       [2.04588666]])

In [28]:
tilde_r_1_example2 # projected messages, influencer 1

[array([[42.52310923],
        [ 8.63576332]]),
 array([[42.57502616],
        [ 8.65646124]]),
 array([[42.57681716],
        [ 8.65717527]]),
 array([[42.57687894],
        [ 8.6571999 ]]),
 array([[42.57688108],
        [ 8.65720075]]),
 array([[42.57688115],
        [ 8.65720078]]),
 array([[42.57688115],
        [ 8.65720078]]),
 array([[42.57688115],
        [ 8.65720078]])]

In [29]:
tilde_r_2_example2 # projected messages, influencer 2

[array([[-51.13677845],
        [-10.529899  ]]),
 array([[-51.09379324],
        [-10.51265073]]),
 array([[-51.09231037],
        [-10.51205571]]),
 array([[-51.09225921],
        [-10.51203519]]),
 array([[-51.09225745],
        [-10.51203448]]),
 array([[-51.09225738],
        [-10.51203445]]),
 array([[-51.09225738],
        [-10.51203445]]),
 array([[-51.09225738],
        [-10.51203445]])]

In [30]:
res2["payoffs"][0] # cumulative payoffs at each stage, influencer 1

[-3990.582453064502,
 -7434.89934027266,
 -10535.288342502681,
 -13325.654097373566,
 -15836.983762754173,
 -18097.180476685804,
 -20131.357519692756,
 -21962.116858413556]

In [31]:
res2["payoffs"][1] # cumulative payoffs at each stage, influencer 2

[-5676.69776600835,
 -10752.188351795472,
 -15319.402854333564,
 -19429.873345054526,
 -23129.296086230832,
 -26458.776531541473,
 -29455.308931645824,
 -32152.188091718777]

# Example 3: $a_1 = 0.9, a_2 = 0.1$

In [32]:
A3 = np.array([
    [0.9, 0.1],
    [0.9, 0.1]
])
V3 = (1/np.sqrt(0.9**2 + 0.1**2)) * np.array([
    [0.9, -0.1],
    [0.1, 0.9]
])

In [33]:
res3 = run_simulation(A3, b1, b2, alpha1, alpha2, delta, c, x0) # compute the solution

In [34]:
tilde_K1_example3 = (V3.T @ res3["K"][0] @ V3) # tilde K1
tilde_k1_example3 = np.linalg.inv(2 * V3 @ tilde_K1_example3) @ (2 * (b1 * np.ones((2, 1))).T @ res3["K"][0] - res3["k"][0]).T # tilde_k1
tilde_K2_example3 = (V3.T @ res3["K"][1] @ V3) # tilde K2
tilde_k2_example3 = np.linalg.inv(2 * V3 @ tilde_K2_example3) @ (2 * (b2 * np.ones((2, 1))).T @ res3["K"][1] - res3["k"][1]).T # tilde_k2
tilde_x_infty_example3 = V3.T @ (res3["x"][0][-1] + 10) # projected limit opinions
tilde_r_1_example3 = [V3.T @ r for r in res3["r"][0]] # projected messages, influencer 1
tilde_r_2_example3 = [V3.T @ r for r in res3["r"][1]] # projected messages, influencer 2

In [35]:
tilde_K1_example3 # tilde_K1

array([[1.03150998, 0.        ],
       [0.        , 1.        ]])

In [36]:
tilde_K2_example3 # tilde_K2

array([[ 1.022195, -0.      ],
       [ 0.      ,  1.      ]])

In [37]:
tilde_k1_example3 # tilde_k1

array([[21.5920095 ],
       [ 8.83452209]])

In [38]:
tilde_k2_example3 # tilde_k2

array([[-16.30100197],
       [ -4.41726104]])

In [39]:
tilde_x_infty_example3 # projected limit opinions

array([[6.14515996],
       [3.45663265]])

In [40]:
tilde_r_1_example3 # projected messages, influencer 1

[array([[42.71159798],
        [14.27419663]]),
 array([[43.01008374],
        [14.51187615]]),
 array([[43.0203023 ],
        [14.52001303]]),
 array([[43.02065213],
        [14.52029159]]),
 array([[43.0206641 ],
        [14.52030113]]),
 array([[43.02066451],
        [14.52030145]]),
 array([[43.02066453],
        [14.52030146]]),
 array([[43.02066453],
        [14.52030146]]),
 array([[43.02066453],
        [14.52030146]])]

In [41]:
tilde_r_2_example3 # projected messages, influencer 2

[array([[-51.88002705],
        [-17.92134819]]),
 array([[-51.63353513],
        [-17.72328192]]),
 array([[-51.62509657],
        [-17.71650118]]),
 array([[-51.62480767],
        [-17.71626905]]),
 array([[-51.62479778],
        [-17.7162611 ]]),
 array([[-51.62479744],
        [-17.71626083]]),
 array([[-51.62479743],
        [-17.71626082]]),
 array([[-51.62479743],
        [-17.71626082]]),
 array([[-51.62479743],
        [-17.71626082]])]

In [42]:
res3["payoffs"][0] # cumulative payoffs at each stage, influencer 1

[-4281.06658216661,
 -8035.67657632715,
 -11418.233051082123,
 -14462.639372216221,
 -17202.608312162378,
 -19668.5804582794,
 -21887.955392870943,
 -23885.39283409842,
 -25683.08653120608]

In [43]:
res3["payoffs"][1] # cumulative payoffs at each stage, influencer 2

[-6250.423855816567,
 -11796.610572538,
 -16783.212574056233,
 -21271.00186819263,
 -25310.007534514392,
 -28945.112489441155,
 -32216.706944414924,
 -35161.14195375389,
 -37811.13346215472]